In [1]:
library(deSolve) # for solving ODE system
library(ggplot2) # for visualisation
library(scales) # y axis values scaling

# SI model
# basic setup 
N   <- 100 
I_0 <- 1
S_0 <- N - I_0
y   <- c(S = S_0, I = I_0)

times  <- seq(from = 0, to = 50, by = 1)  # time step 
params <- c(beta = 0.4)

si_model <- function(times, state, parms)
    {
    S <- state["S"]
    I <- state["I"]
    R <- state["R"]
    N <- S + I
    beta <- parms["beta"]
    dS <- - (beta * I / N) * S
    dI <- (beta * I / N) * S
    res <- list(c(dS, dI))
    return(res)
    }

In [ ]:
# SIR model function
sir_model <- function(time, state, parameters) {
  with(as.list(c(state, parameters)), {
    dS <- -beta * S * I / N
    dI <- beta * S * I / N - gamma * I
    dR <- gamma * I
    return(list(c(dS, dI, dR)))
  })
}

In [ ]:
# solve equations
output_raw <- ode(y = y, times = times, func = si_model, parms = params)

# convert matrix to dataframe
output <- as.data.frame(output_raw)

# plot (without using ggplot2)
plot(output$time, output$S, type = "l", col = "blue", lwd = 2, ylim = c(0, N),
     xlab = "Time", ylab = "Number")
lines(output$time, output$I, lwd = 2, col = "red", type = "l")
legend("topright", legend = c("Susceptible", "Infectious"),
       col = c("blue", "red"), lwd = 2, bty = "n")

In [ ]:
# Set parameters
parameters <- c(beta = 0.5, gamma = 0.1)
state <- c(S = 99000, I = 1000, R = 0)
times <- seq(0, 180, by = 1)
N <- sum(state)

In [ ]:
# Solve the differential equations
out <- ode(y = state, times = times, func = sir_model, parms = parameters)
df <- as.data.frame(out)

In [ ]:
# Plot
ggplot(df, aes(x = time)) +
  geom_line(aes(y = S, color = "Susceptible")) +
  geom_line(aes(y = I, color = "Infected")) +
  geom_line(aes(y = R, color = "Recovered")) +
  scale_color_manual(values = c("Susceptible" = "blue", "Infected" = "red", "Recovered" = "green")) +
  labs(x = "Time (Days)", y = "Number of infections", color = "") +
  theme_minimal() +
  theme(legend.position = "right")+
  scale_y_continuous(labels = comma)

In [ ]:
# seir model 
# define the model fucntion first 
seir_model <- function(times, state, parms)
    {
    S <- state["S"]
    E <- state["E"]
    I <- state["I"]
    R <- state["R"]
    N <- S + E + I + R

    beta  <- parms["beta"]
    delta <- parms["delta"]
    gamma <- parms["gamma"]

    dS  <- -(beta * I / N) * S
    dE  <- (beta * I / N) * S - delta * E
    dI  <- delta * E - gamma * I
    dR  <- gamma * I
    res <- list(c(dS, dE, dI, dR))
    return (res)
}

# define params
parms <- c(beta = 0.4, delta = 0.2, gamma = 0.2)

# define times
times <- seq(from = 0, to = 100, by = 1)

# Define initial conditions
N <- 100
E_0 <- 0
I_0 <- 1
R_0 <- 0
S_0 <- N - E_0 - I_0 - R_0
y <- c(S = S_0, E = E_0, I = I_0, R = R_0)

# Solve equations
output_raw <- ode(y = y, times = times, func = seir_model, parms = parms)

# Convert matrix to data frame for easier manipulation
output <- as.data.frame(output_raw)

In [ ]:
# Plot model output
plot(output$time, output$S, type = "l", col = "blue", lwd = 2, ylim = c(0, N),
      xlab = "Time", ylab = "Number")
lines(output$time, output$E, lwd = 2, col = "yellow", type = "l")
lines(output$time, output$I, lwd = 2, col = "red", type = "l")
lines(output$time, output$R, lwd = 2, col = "green", type = "l")
legend("topright", legend = c("Susceptible", "Exposed", "Infectious", "Recovered"),
       col = c("blue", "yellow", "red", "green"), lwd = 2, bty = "n")

In [ ]:
# seirv model
seirv_model <- function(times, state, parms) {
    S <- state["S"]
    E <- state["E"]
    I <- state["I"]
    R <- state["R"]
    V <- state["V"]
    N <- S + E + I + R + V
    
    beta <- parms["beta"]
    delta <- parms["delta"]
    gamma <- parms["gamma"]
    v <- parms["v"]
    
    dS <- -(beta * I / N) * S - v * S
    dE <- (beta * I / N) * S - delta * E
    dI <- delta * E - gamma * I
    dR <- gamma * I
    dV <- v * S
    
    return(list(c(dS, dE, dI, dR, dV)))
}

# Parameter values
parms <- c(beta = 0.4, delta = 0.2, gamma = 0.2, v = 0.2)  # 테스트용 v ↑

# Time steps
times <- seq(from = 0, to = 100, by = 1)

# Initial conditions
N <- 100
E_0 <- 0
I_0 <- 1
R_0 <- 0
V_0 <- 0
S_0 <- N - E_0 - I_0 - R_0 - V_0
y <- c(S = S_0, E = E_0, I = I_0, R = R_0, V = V_0)

# Solve ODE
output_raw <- ode(y = y, times = times, func = seirv_model, parms = parms)
output <- as.data.frame(output_raw)


In [ ]:
# Plot
plot(output$time, output$S, type = "l", col = "blue", lwd = 2, ylim = c(0, N),
     xlab = "Time", ylab = "Number")
lines(output$time, output$E, col = "yellow", lwd = 2)
lines(output$time, output$I, col = "red", lwd = 2)
lines(output$time, output$R, col = "green", lwd = 2)
lines(output$time, output$V, col = "purple", lwd = 2)
legend("topright", legend = c("Susceptible", "Exposed", "Infectious", "Recovered", "Vaccinated"),
       col = c("blue", "yellow", "red", "green", "purple"), lwd = 2, bty = "n")


In [ ]:
# sirv model with vaccination start time, coverage, duration

sirv_model <- function(time, state, parameters) {
  with(as.list(c(state, parameters)), {
    
    N <- S + I + R + V
    
    vacc_rate <- ifelse(time >= vaccine_start,
                        vaccine_coverage / vaccine_duration,
                        0)
    
    dS <- -beta * S * I / N - (vacc_rate * vacc_eff) * S
    dI <- beta * S * I / N - gamma * I
    dR <- gamma * I
    dV <- (vacc_rate * vacc_eff) * S
    
    return(list(c(dS, dI, dR, dV)))
  })
}

In [ ]:
initial_state <- c(
  S = 99000,
  I = 1000,
  R = 0,
  V = 0
)

times <- seq(0, 180, by = 1)

# different start days
start_days <- c(10, 30, 50, 70)


In [ ]:
# for loop
all_outputs <- data.frame()
for(i in start_days) {
  
  parameters <- c(
    beta = 0.3,
    gamma = 1/7,
    vaccine_start = i,
    vaccine_coverage = 0.5,
    vaccine_duration = 14,
    vacc_eff = 0.7
  )
  
  out <- as.data.frame(ode(y = initial_state, times = times, func = sirv_model, parms = parameters))
  
  all_outputs <- rbind(all_outputs, 
                       data.frame(time = out$time,
                                  I = out$I,
                                  vaccine_start = i))
}
all_outputs$type <- "With vaccination"

# bind with pre-vacc
pre_vacc <- df[,c(1,3)]
pre_vacc$vaccine_start <- 0 
pre_vacc$type <- "Without vaccination"
all_outputs <- rbind(pre_vacc, all_outputs)

In [ ]:
ggplot(all_outputs)+
  geom_line(aes(x = time, y = I, color = as.factor(vaccine_start), linetype = as.factor(type)))+
  theme_bw()+
  labs(color = "Vaccine start day",
      linetype = "Type")+
  scale_y_continuous(labels = comma)+
  xlab("Time (Days)")+
  ylab("Number of infections")

In [ ]:
# differnt coverage
vacc_coverage <- c(0.1, 0.3, 0.5, 0.7)

# for loop
all_outputs <- data.frame()
for(vc in vacc_coverage) {
  
  parameters <- c(
    beta = 0.3,
    gamma = 1/7,
    vaccine_start = 10,
    vaccine_coverage = vc,
    vaccine_duration = 14,
    vacc_eff = 0.7
  )
  
  out <- as.data.frame(ode(y = initial_state, times = times, func = sirv_model, parms = parameters))
  
  all_outputs <- rbind(all_outputs, 
                       data.frame(time = out$time,
                                  I = out$I,
                                  vaccine_coverage = vc))
}
all_outputs$type <- "With vaccination"
colnames(pre_vacc)[3] <- "vaccine_coverage"
all_outputs <- rbind(pre_vacc, all_outputs)

In [ ]:
ggplot(all_outputs)+
  geom_line(aes(x = time, y = I, color = as.factor(vaccine_coverage) , linetype = as.factor(type)))+
  theme_bw()+
  scale_y_continuous(labels = comma)+
  labs(color = "Vaccine coverage",
       linetype = "Type") +
  xlab("Time (Days)")+
  ylab("Number of infections")

In [ ]:
vacc_eff <- c(0.1, 0.3, 0.5, 0.7, 0.9)

# for loop
all_outputs <- data.frame()
for(ve in vacc_eff) {
  
  parameters <- c(
    beta = 0.3,
    gamma = 1/7,
    vaccine_start = 10,
    vaccine_coverage = 0.5,
    vaccine_duration = 14,
    vacc_eff         = ve
  )
  
  out <- as.data.frame(ode(y = initial_state, times = times, func = sirv_model, parms = parameters))
  
  all_outputs <- rbind(all_outputs, 
                       data.frame(time = out$time,
                                  I = out$I,
                                  vacc_eff = ve))
}
all_outputs$type <- "With vaccination"
colnames(pre_vacc)[3] <- "vacc_eff"
all_outputs <- rbind(pre_vacc, all_outputs)

In [ ]:
ggplot(all_outputs)+
  geom_line(aes(x = time, y = I, color = as.factor(vacc_eff) , linetype = as.factor(type)))+
  theme_bw()+
  scale_y_continuous(labels = comma)+
  labs(color = "Vaccine efficacy",
       linetype = "Type") +
  xlab("Time (Days)")+
  ylab("Number of infections")

In [ ]:
# Age structured SEIRV model with daily vaccination rate (v) and vaccine efficacy (VE)
seirv_age_model <- function(times, state, parms) {
  with(as.list(c(state, parms)), {
    
    # Effective daily vaccination rate: vaccination starts only after day T (here, 10 days and 30 days after)
    v1_eff <- ifelse(times >= 5, parms["v1"], 0)
    v2_eff <- ifelse(times >= 50, parms["v2"], 0)

    # Age group 1
    N1 <- S1 + E1 + I1 + R1 + V1
    dS1 <- -(beta1 * I1 / N1) * S1 - v1_eff * S1 # you vaccinate v1_eff (%) among the S 
    dE1 <- (beta1 * I1 / N1) * S1 - delta * E1
    dI1 <- delta * E1 - gamma * I1
    dR1 <- gamma * I1
    dV1 <- v1_eff * S1 * VE1    # among the total "raw" vaccinated, only successfully vaccinated (Vaccine efficacy) people are moved to V

    # Age group 2
    N2 <- S2 + E2 + I2 + R2 + V2
    dS2 <- -(beta2 * I2 / N2) * S2 - v2_eff * S2
    dE2 <- (beta2 * I2 / N2) * S2 - delta * E2
    dI2 <- delta * E2 - gamma * I2
    dR2 <- gamma * I2
    dV2 <- v2_eff * S2 * VE2    

    return(list(c(dS1, dE1, dI1, dR1, dV1,
                  dS2, dE2, dI2, dR2, dV2)))
  })
}

# Parameters (can differ by age group)
parms <- c(
  beta1 = 0.5, v1 = 0.01, VE1 = 0.8,   # group 1: daily rate 0.01, efficacy 90%
  beta2 = 0.5, v2 = 0.01, VE2 = 0.8,  # group 2: daily rate 0.02, efficacy 85%
  delta = 0.2,
  gamma = 0.2
)

# Initial conditions
N1 <- 1000
N2 <- 1000
init <- c(
  S1 = N1 - 1, E1 = 0, I1 = 1, R1 = 0, V1 = 0,
  S2 = N2 - 1, E2 = 0, I2 = 1, R2 = 0, V2 = 0
)

times <- seq(0, 100, by = 1)

# Solve ODE
out <- ode(y = init, times = times, func = seirv_age_model, parms = parms)
df <- as.data.frame(out)


In [ ]:
# Infectious individuals over time
par(mfrow = c(2, 1))
plot(df$time, df$I1, type = "l", col = "red", lwd = 2, ylim = c(0, max(df$I1, df$I2)),
     main = "Infectious - Age Group 1", xlab = "Time", ylab = "I1")
plot(df$time, df$I2, type = "l", col = "blue", lwd = 2, ylim = c(0, max(df$I1, df$I2)),
     main = "Infectious - Age Group 2", xlab = "Time", ylab = "I2")

In [ ]:
par(mfrow = c(2, 1), mar = c(4, 4, 2, 2)) 

# === Age Group 1 ===
plot(df$time, df$S1, type = "l", col = "skyblue", lwd = 2,
     xlab = "Time", ylab = "Population", main = "SEIRV - Age Group 1",
     ylim = c(0, max(df$S1, df$E1, df$I1, df$R1, df$V1)))
lines(df$time, df$E1, col = "orange", lwd = 2)
lines(df$time, df$I1, col = "red", lwd = 2)
lines(df$time, df$R1, col = "green", lwd = 2)
lines(df$time, df$V1, col = "purple", lwd = 2)
legend("right", legend = c("S", "E", "I", "R", "V"),
       col = c("skyblue", "orange", "red", "green", "purple"),
       lty = 1, lwd = 2, bty = "n")

# === Age Group 2 ===
plot(df$time, df$S2, type = "l", col = "skyblue", lwd = 2,
     xlab = "Time", ylab = "Population", main = "SEIRV - Age Group 2",
     ylim = c(0, max(df$S2, df$E2, df$I2, df$R2, df$V2)))
lines(df$time, df$E2, col = "orange", lwd = 2)
lines(df$time, df$I2, col = "red", lwd = 2)
lines(df$time, df$R2, col = "green", lwd = 2)
lines(df$time, df$V2, col = "purple", lwd = 2)
legend("right", legend = c("S", "E", "I", "R", "V"),
       col = c("skyblue", "orange", "red", "green", "purple"),
       lty = 1, lwd = 2, bty = "n")